## Vistual inertia scheduling

In [1]:
import andes
import os

from statistics import fmean
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import gurobipy as gb
import pandas as pd
import numpy as np
import logging

logger = logging.getLogger(__name__)

# from opf import dcopf
from visopf import vis1

## Main - IEEE 39


### load case from andes excel

In [2]:
# get andes case from excel
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee39_vis.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

# ssa.config.mva = 1000

ssa.config.mva

100

### load norm parameter

In [3]:
# prepare nn data for visopf
data_path = dir_path + '/VIS_opf/NN_train'

fnorm = pd.read_csv(data_path + '/fnorm.csv')
pnorm = pd.read_csv(data_path + '/pnorm.csv')
norm = {'fnorm': fnorm, 'pnorm': pnorm }
norm


{'fnorm':            M         D         Fg          Rg    fnadir
 0  14.988180  7.492874  80.045126  124.959573 -0.013860
 1   2.891501  1.438628  40.350784   43.546600  0.006387,
 'pnorm':            M         D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  14.988180  7.492874  80.045126  124.959573  4.018493  2.004602  0.143188
 1   2.891501  1.438628  40.350784   43.546600  2.317657  1.151364  0.086313}

In [4]:
# example to call norm data
norm['fnorm']['M'].iloc[0] # mean
norm['fnorm']['M'].iloc[1] # std 

2.891501173118185

### load nn parameter

In [5]:
fw1 = pd.read_csv(data_path + '/fw1.csv', header=None)
fw2 = pd.read_csv(data_path + '/fw2.csv', header=None)

fb1 = pd.read_csv(data_path + '/fb1.csv', header=None)
fb2 = pd.read_csv(data_path + '/fb2.csv', header=None)

In [6]:
pw1 = pd.read_csv(data_path + '/pw1.csv', header=None)
pw2 = pd.read_csv(data_path + '/pw2.csv', header=None)

pb1 = pd.read_csv(data_path + '/pb1.csv', header=None)
pb2 = pd.read_csv(data_path + '/pb2.csv', header=None)

In [7]:
nn = {
        'fw1': fw1,
        'fw2': fw2,       
        'fb1': fb1,
        'fb2': fb2,
        'pw1': pw1,
        'pw2': pw2,
        'pb1': pb1,
        'pb2': pb2,
    }

### test opf model

In [8]:
ss = vis1(norm=norm, nn=nn, dpe=0.01)
ss.norm

Restricted license - for non-production use only - expires 2023-10-25


{'fnorm':            M         D         Fg          Rg    fnadir
 0  14.988180  7.492874  80.045126  124.959573 -0.013860
 1   2.891501  1.438628  40.350784   43.546600  0.006387,
 'pnorm':            M         D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  14.988180  7.492874  80.045126  124.959573  4.018493  2.004602  0.143188
 1   2.891501  1.438628  40.350784   43.546600  2.317657  1.151364  0.086313}

In [9]:
ss.norm['fnorm']['M'].iloc[0]

14.988179599999995

In [10]:
ss.nn['fw1']

,0,1,2,3
0,0.007024,-0.009032,-0.003725,0.000279
1,0.011937,-0.004678,0.104979,0.009249
2,-0.021316,0.001189,-0.056554,-0.015670
3,-0.012752,-0.000368,-0.006217,0.004011
4,-0.015508,0.020023,-0.125958,-0.015702
...,...,...,...,...
59,-0.002516,0.007691,0.011266,0.005049
60,-0.036352,0.001531,-0.069906,-0.031887
61,-0.023868,0.009889,-0.104516,-0.015767
62,-0.006000,0.003099,-0.005233,-0.009474


In [11]:
ss.nn['fw1'][1].iloc[1]
# ss.nn['fb1']

-0.004678414

In [12]:
vsg_ieee14 = ['PV_6', 'PV_7']
vsg_ieee39 = ['PV_1', 'PV_6', 'PV_8', 'PV_9']

ss.from_andes(ssa, vsg_ieee39, Sbase=1000)

ss.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,...,ramp30,type,p_pre,band,K,M,D,R,Mvsg,Dvsg
0,PV_1,1.0,PV_1,10.400,34.5,30,4.360864,15.0,2.0,1.035534,...,600,2,0,13.0,1,0.000000,0.0000,0.000000,10.4000,5.2000
1,PV_2,1.0,PV_2,8.360,34.5,31,6.460000,9.0,1.5,1.013246,...,600,1,0,7.5,1,6.738160,0.8360,0.059809,0.0000,0.0000
2,PV_3,1.0,PV_3,8.437,21.0,32,7.250000,8.0,1.0,1.020528,...,600,1,0,7.0,1,6.040892,0.8437,0.059263,0.0000,0.0000
3,PV_4,1.0,PV_4,11.748,21.0,33,6.520000,7.0,1.0,1.013430,...,600,1,0,6.0,1,10.244256,1.1748,0.042560,0.0000,0.0000
4,PV_5,1.0,PV_5,10.802,15.5,34,5.200000,7.0,1.0,1.019109,...,600,1,0,6.0,1,5.617040,1.0802,0.046288,0.0000,0.0000
5,PV_6,1.0,PV_6,10.857,15.5,35,6.900000,8.0,1.0,1.060000,...,600,2,0,7.0,1,0.000000,0.0000,0.000000,6.5142,5.4285
6,PV_7,1.0,PV_7,10.252,12.5,36,5.900000,7.0,1.0,1.060000,...,600,1,0,6.0,1,8.488656,1.0252,0.048771,0.0000,0.0000
7,PV_8,1.0,PV_8,9.702,12.5,37,3.300000,7.0,1.0,1.013996,...,600,2,0,6.0,1,0.000000,0.0000,0.000000,7.7616,2.9106
8,PV_9,1.0,PV_9,16.841,34.5,38,7.800000,10.0,1.0,1.052803,...,600,2,0,9.0,1,0.000000,0.0000,0.000000,10.1046,8.4205
9,Slack_10,1.0,Slack_10,11.990,345.0,39,5.741700,15.0,3.0,1.030000,...,600,1,0,12.0,1,11.990000,1.1990,0.041701,0.0000,0.0000


In [13]:
ss.norm

{'fnorm':            M         D         Fg          Rg    fnadir
 0  14.988180  7.492874  80.045126  124.959573 -0.013860
 1   2.891501  1.438628  40.350784   43.546600  0.006387,
 'pnorm':            M         D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  14.988180  7.492874  80.045126  124.959573  4.018493  2.004602  0.143188
 1   2.891501  1.438628  40.350784   43.546600  2.317657  1.151364  0.086313}

In [14]:
a_test = []
for i in range(64):
    a_test.append('a'+str(i))
# a_test

In [15]:
ss.gendict.keys()

dict_keys(['PV_1', 'PV_2', 'PV_3', 'PV_4', 'PV_5', 'PV_6', 'PV_7', 'PV_8', 'PV_9', 'Slack_10'])

In [16]:
ss.nn['fw2'][4].iloc[0]

-0.13017723

In [17]:
ss.nn['fb1']

,0
0,-0.290703
1,0.202561
2,0.151374
3,-0.195499
4,0.256226
...,...
59,-0.389984
60,0.222762
61,0.228741
62,-0.170625


In [18]:
ss.mva

100

In [19]:
ss.nn['fb2'][0].iloc[0]

0.26405954

In [20]:
ss.build()

# ss.mdl.optimize()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.


In [21]:
ss.get_res()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 264 rows, 166 columns and 2790 nonzeros
Model fingerprint: 0x60599787
Variable types: 102 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [9e-08, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [4e-04, 1e+02]
Found heuristic solution: objective 58.5640000
Presolve removed 210 rows and 132 columns
Presolve time: 0.00s
Presolved: 54 rows, 34 columns, 411 nonzeros
Variable types: 23 continuous, 11 integer (11 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

So

Successfully solve vis1.


(        gen         pg  pru  prd
 0      PV_1   2.000000  0.0  0.0
 1      PV_2   5.574803  0.0  0.0
 2      PV_3   6.952996  0.0  0.0
 3      PV_4   7.000000  0.0  0.0
 4      PV_5   4.664157  0.0  0.0
 5      PV_6   7.135843  0.0  0.0
 6      PV_7   7.000000  0.0  0.0
 7      PV_8   3.725946  0.0  0.0
 8      PV_9   1.000000  0.0  0.0
 9  Slack_10  13.510255  0.0  0.0,
     gen      Mvsg      Dvsg
 0  PV_1  8.000000  5.000000
 1  PV_6  0.000000  5.000000
 2  PV_8  0.447529  0.001525
 3  PV_9  0.000000  5.000000)

In [22]:
ss.load.p0.sum()


58.56399999999999

In [23]:
ss.costdict

{'PV_1': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_2': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_3': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_4': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_5': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_6': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_7': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_8': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_9': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'Slack_10': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0}}

In [24]:
ss.mdl.display()

Minimize
<gurobi.LinExpr: pg[PV_1] + pg[PV_2] + pg[PV_3] + pg[PV_4] + pg[PV_5] + pg[PV_6]
+ pg[PV_7] + pg[PV_8] + pg[PV_9] + pg[Slack_10]>
Subject To
PowerBalance: <gurobi.LinExpr: pg[PV_1] + pg[PV_2] + pg[PV_3] + pg[PV_4] + pg[PV_5] +
 pg[PV_6] + pg[PV_7] + pg[PV_8] + pg[PV_9] + pg[Slack_10]> = 58.564
Line_1_U: <gurobi.LinExpr: -0.5607822357546101 pg[PV_1] + -0.6089602208400838 pg[PV_2]
+ -0.3983097910057284 pg[PV_3] + -0.4219560834463756 pg[PV_4] + -0.5066725368514864
pg[PV_5] + -0.5066725368514864 pg[PV_6] + -0.5066725368514856 pg[PV_7] +
-0.5066725368514856 pg[PV_8] + -0.5988298865678324 pg[PV_9] + -0.5607822357546101
 pg[Slack_10]> <= -20.9585
Line_1_D: <gurobi.LinExpr: -0.5607822357546101 pg[PV_1] + -0.6089602208400838 pg[PV_2]
+ -0.3983097910057284 pg[PV_3] + -0.4219560834463756 pg[PV_4] + -0.5066725368514864
pg[PV_5] + -0.5066725368514864 pg[PV_6] + -0.5066725368514856 pg[PV_7] +
-0.5066725368514856 pg[PV_8] + -0.5988298865678324 pg[PV_9] + -0.5607822357546101
 pg[Slack_10]> >=

In [25]:
ss.Mvsg['PV_1'].X

8.0

In [26]:
ss.Mvsg['PV_8'].X

0.4475287283502425

In [27]:
ss.Mvsg['PV_6'].X

0.0

In [28]:
ss.Mvsg['PV_9'].X

0.0